Copyright (c) 2019 [윤기태]

https://github.com/yoonkt200/python-data-analysis

[MIT License](https://github.com/yoonkt200/python-data-analysis/blob/master/LICENSE.txt)

# (가제) 파이썬 데이터 분석

-----

# 4.2) 강남역 맛집 리뷰로 알아보는 감성분석 

### 바로가기

- [<Step1. 크롤링> : 네이버 플레이스 리뷰 크롤링](#<Step1.-크롤링>-:-네이버-플레이스-리뷰-크롤링)
    - [강남역 맛집 리스트 가져오기]
    - [맛집 리뷰 정보 크롤링]
- [<Step2. 분석> : TF-IDF를 이용한 핵심어 추출](#<Step2.-분석>-:-TF\-IDF를-이용한-핵심어-추출)
    - []
- [<Step3. 분류> : 긍정 부정 리뷰 분류하기](#<Step3.-분류>-:-긍정-부정-리뷰-분류하기)
    - []
- [<Step4. 평가> : 모델 성능과 키워드 분석](#<Step4.-평가>-:-모델-성능과-키워드-분석)
    - []

-----

In [103]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# <Step1. 크롤링> : 네이버 플레이스 리뷰 크롤링

### [강남역 맛집 리스트 가져오기]

In [163]:
# # 관련도순 상위 100개의 고기집 리스트 정보
# import requests
# import json

# source_url = "https://store.naver.com/sogum/api/businesses?"
# url_parameter_start = "start=1"
# url_parameter_display = "&display=" # 번호 입력 필요
# url_parameter_query = "&query=강남역+고기집"
# url_parameter_sorting = "&sortingOrder=precision"
# url_concat = source_url + url_parameter_start + \
#             url_parameter_display + str(100) + url_parameter_query + url_parameter_sorting
# json_data = requests.get(url_concat).text
# restaurant_list_data = json.loads(json_data)
# print(restaurant_list_data)

In [164]:
# # 크롤링에 필요한 id 정보 추출
# restaurant_id_list = []

# for start_idx in [1, 101, 201, 301, 401, 501, 601, 701, 801, 901]:
#     source_url = "https://store.naver.com/sogum/api/businesses?"
#     url_parameter_start = "start=" + str(start_idx)
#     url_parameter_display = "&display="
#     url_parameter_query = "&query=강남역+고기집"
#     url_parameter_sorting = "&sortingOrder=precision"
#     url_concat = source_url + url_parameter_start + \
#                 url_parameter_display + str(start_idx+99) + url_parameter_query + url_parameter_sorting
#     print("request_url:", url_concat)
#     json_data = requests.get(url_concat).text
#     restaurant_list_data = json.loads(json_data)

#     for restaurant in restaurant_list_data['items']:
#         if 'moreBookingReviewsPath' in restaurant:
#             restaurant_id_list.append(restaurant['id'])
#
# restaurant_id_list = list(set(restaurant_id_list))

-----

### [맛집 리뷰 정보 크롤링]

In [175]:
# # 가져온 맛집 리스트의 리뷰 정보 크롤링
# import requests
# from bs4 import BeautifulSoup
# import re

# columns = ['score', 'review']
# df = pd.DataFrame(columns=columns)

# source_url_head = "https://store.naver.com/restaurants/detail?id="
# source_url_tail = "&tab=bookingReview#_tab"

# for idx in range(0, len(restaurant_id_list)):
#     print("Crawl", str(int(idx/len(restaurant_id_list)*100)), "% complete..")
#     req = requests.get(source_url_head + str(restaurant_id_list[idx]) + source_url_tail)
#     html = req.content
#     soup = BeautifulSoup(html, 'lxml')
#     review_area = soup.find(name="div", attrs={"class":"review_area"})

#     # 리뷰가 없는 페이지
#     if review_area is None:
#         continue

#     review_list = review_area.find_all(name="div", attrs={"class":"info_area"})
#     for review in review_list:
#         score = review.find(name="span", attrs={"class":"score"}).text
#         review_txt = review.find(name="div", attrs={"class":"review_txt"}).text

#         # to dataframe
#         row = [score, review_txt]
#         series = pd.Series(row, index=df.columns)
#         df = df.append(series, ignore_index=True)
# print("Crawl 100 %", "complete")

In [176]:
# # 4점 이상의 리뷰는 긍정 리뷰, 3점 이하의 리뷰는 부정 리뷰로 평가
# df['y'] = df['score'].apply(lambda x: 1 if int(x) > 3 else 0)
# del df['score']
# print(df.shape)
# df.head()

In [178]:
df = pd.read_csv("temp_data.csv")

In [179]:
print(df.shape)
df.head()

(540, 2)


,review,y
0,불판이 작아서 6명 테이블에 6명이 고기먹기엔 힘들었어요 ㅠㅠ,0
1,좋았습니다,1
2,분위기도 좋고 했지만 고기가 얇아 아쉬웠습니다.,1
3,생삼겹 고기 맛이 별로입니다.너무 얇음;;\n이베리코 목살은 돼지 냄새나요;;;,0
4,고기가맛있어서. . . 저도모르게그만 술을...,1


-----

# <Step2. 분석> : TF-IDF를 이용한 핵심어 추출